In [1]:
import numpy as np

In [3]:
# Spark context `sc` is already loaded so we don't need to load it!
# Importing SparkSession

from pyspark.sql import SparkSession

NameError: name 'sc' is not defined

In [4]:
# Creating a SparkSession object - spark
spark = SparkSession.builder.getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.NumberFormatException: Size must be specified as bytes (b), kibibytes (k), mebibytes (m), gibibytes (g), tebibytes (t), or pebibytes(p). E.g. 50b, 100k, or 250m.
Failed to parse byte string: --driver-memory
	at org.apache.spark.network.util.JavaUtils.byteStringAs(JavaUtils.java:243)
	at org.apache.spark.network.util.JavaUtils.byteStringAsBytes(JavaUtils.java:254)
	at org.apache.spark.util.Utils$.byteStringAsBytes(Utils.scala:1073)
	at org.apache.spark.SparkConf.getSizeAsBytes(SparkConf.scala:294)
	at org.apache.spark.memory.UnifiedMemoryManager$.getMaxMemory(UnifiedMemoryManager.scala:218)
	at org.apache.spark.memory.UnifiedMemoryManager$.apply(UnifiedMemoryManager.scala:194)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:308)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:165)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:256)
	at org.a

In [69]:
train = spark.read.csv('s3n://sparkstoragetest/tr2.csv', inferSchema= True, header = True)

In [70]:
test = spark.read.csv('s3n://sparkstoragetest/te2.csv', inferSchema= True, header = True)

In [71]:
train.show()

+------+--------+-----------------+-----------------+---------------+------------------+-------------+----+---+------+--------------+-----------+----------+------------+-----------+------------------+-----------------+-------------------+----------------------+---------------------+----------------+
|  msno|is_churn|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|city| bd|gender|registered_via|trans_month|trans_year|expire_month|expire_year|registration_month|registration_year|membership_duration|autorenew_._not_cancel|registration_duration|reg_mem_duration|
+------+--------+-----------------+-----------------+---------------+------------------+-------------+----+---+------+--------------+-----------+----------+------------+-----------+------------------+-----------------+-------------------+----------------------+---------------------+----------------+
|280470|       0|               41|               30|            149|               149|         

In [72]:
train.head()

Row(msno=280470, is_churn=0, payment_method_id=41, payment_plan_days=30, plan_list_price=149, actual_amount_paid=149, is_auto_renew=1, city=5, bd=21, gender=0, registered_via=7, trans_month=12, trans_year=2016, expire_month=1, expire_year=2017, registration_month=3, registration_year=2015, membership_duration=31, autorenew_._not_cancel=1, registration_duration=672, reg_mem_duration=641)

In [73]:
test.show()

+------+--------+-----------------+-----------------+---------------+------------------+-------------+----+---+------+--------------+-----------+----------+------------+-----------+------------------+-----------------+-------------------+----------------------+---------------------+----------------+
|  msno|is_churn|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|city| bd|gender|registered_via|trans_month|trans_year|expire_month|expire_year|registration_month|registration_year|membership_duration|autorenew_._not_cancel|registration_duration|reg_mem_duration|
+------+--------+-----------------+-----------------+---------------+------------------+-------------+----+---+------+--------------+-----------+----------+------------+-----------+------------------+-----------------+-------------------+----------------------+---------------------+----------------+
|245780|       0|               38|               30|            149|               149|         

In [74]:
train.printSchema()

root
 |-- msno: integer (nullable = true)
 |-- is_churn: integer (nullable = true)
 |-- payment_method_id: integer (nullable = true)
 |-- payment_plan_days: integer (nullable = true)
 |-- plan_list_price: integer (nullable = true)
 |-- actual_amount_paid: integer (nullable = true)
 |-- is_auto_renew: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- bd: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- registered_via: integer (nullable = true)
 |-- trans_month: integer (nullable = true)
 |-- trans_year: integer (nullable = true)
 |-- expire_month: integer (nullable = true)
 |-- expire_year: integer (nullable = true)
 |-- registration_month: integer (nullable = true)
 |-- registration_year: integer (nullable = true)
 |-- membership_duration: integer (nullable = true)
 |-- autorenew_._not_cancel: integer (nullable = true)
 |-- registration_duration: integer (nullable = true)
 |-- reg_mem_duration: integer (nullable = true)



In [75]:
train.head(1)

[Row(msno=280470, is_churn=0, payment_method_id=41, payment_plan_days=30, plan_list_price=149, actual_amount_paid=149, is_auto_renew=1, city=5, bd=21, gender=0, registered_via=7, trans_month=12, trans_year=2016, expire_month=1, expire_year=2017, registration_month=3, registration_year=2015, membership_duration=31, autorenew_._not_cancel=1, registration_duration=672, reg_mem_duration=641)]

In [76]:
train.columns

['msno',
 'is_churn',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via',
 'trans_month',
 'trans_year',
 'expire_month',
 'expire_year',
 'registration_month',
 'registration_year',
 'membership_duration',
 'autorenew_._not_cancel',
 'registration_duration',
 'reg_mem_duration']

In [77]:
assembler_bhai = VectorAssembler(inputCols = ['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via',
 'trans_month',
 'trans_year',
 'expire_month',
 'expire_year',
 'registration_month',
 'registration_year',
 'membership_duration',
 'registration_duration',
 'reg_mem_duration'], outputCol = 'features')

In [78]:
output_bhai = assembler_bhai.transform(train)

In [79]:
final_data_bhai = output_bhai.select('features', 'is_churn')

In [80]:
train_is_churn, test_is_churn = final_data_bhai.randomSplit([0.7,0.3])

In [81]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr_churn = LogisticRegression(labelCol = 'is_churn')

In [52]:
traindf, train_test_df = train.randomSplit([0.7,0.3])

In [55]:
train = VectorAssembler(inputCols = ['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via',
 'trans_month',
 'trans_year',
 'expire_month',
 'expire_year',
 'registration_month',
 'registration_year',
 'membership_duration',
 'registration_duration',
 'reg_mem_duration'], outputCol = "features").transform(train)

In [56]:
rf = RandomForestClassifier(labelCol = "is_churn", featuresCol = "features")

In [63]:
rf.summary

AttributeError: 'RandomForestClassifier' object has no attribute 'summary'

In [59]:
model = rf.fit(traindf)

IllegalArgumentException: u'Field "features" does not exist.'

In [10]:
from pyspark.ml.feature import VectorAssembler

In [40]:
assembler = VectorAssembler(inputCols = ['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via',
 'trans_month',
 'trans_year',
 'expire_month',
 'expire_year',
 'registration_month',
 'registration_year',
 'membership_duration',
 'registration_duration',
 'reg_mem_duration'], outputCol = 'features')

In [42]:
output_traine = assembler.transform(train)

IllegalArgumentException: u'Output column features already exists.'

In [33]:
output_train

DataFrame[msno: int, is_churn: int, payment_method_id: int, payment_plan_days: int, plan_list_price: int, actual_amount_paid: int, is_auto_renew: int, city: int, bd: int, gender: int, registered_via: int, trans_month: int, trans_year: int, expire_month: int, expire_year: int, registration_month: int, registration_year: int, membership_duration: int, autorenew_._not_cancel: int, registration_duration: int, reg_mem_duration: int, features: vector]

In [34]:
output_test = assembler.transform(test)

In [35]:
output_test

DataFrame[msno: int, is_churn: int, payment_method_id: int, payment_plan_days: int, plan_list_price: int, actual_amount_paid: int, is_auto_renew: int, city: int, bd: int, gender: int, registered_via: int, trans_month: int, trans_year: int, expire_month: int, expire_year: int, registration_month: int, registration_year: int, membership_duration: int, autorenew_._not_cancel: int, registration_duration: int, reg_mem_duration: int, features: vector]

In [39]:
output_train.printSchema()

NameError: name 'output_train' is not defined

In [37]:
# Feature assembler as a vector
train = VectorAssembler(inputCols=['msno',
 'payment_method_id',
 'payment_plan_days',
 'plan_list_price',
 'actual_amount_paid',
 'is_auto_renew',
 'city',
 'bd',
 'gender',
 'registered_via',
 'trans_month',
 'trans_year',
 'expire_month',
 'expire_year',
 'registration_month',
 'registration_year',
 'membership_duration',
 'registration_duration',
 'reg_mem_duration'],outputCol="features").transform(train)
 
rf = RandomForestClassifier(labelCol="is_churn", featuresCol="features")
 
model = rf.fit(train)
 
predictions = model.transform(train)
 
# Select example rows to display.
predictions.select(col("prediction"),col("probability"),).show(5)


Py4JJavaError: An error occurred while calling o276.fit.
: java.lang.ClassCastException: java.lang.Integer cannot be cast to java.lang.Double
	at scala.runtime.BoxesRunTime.unboxToDouble(BoxesRunTime.java:114)
	at org.apache.spark.sql.Row$class.getDouble(Row.scala:242)
	at org.apache.spark.sql.catalyst.expressions.GenericRow.getDouble(rows.scala:192)
	at org.apache.spark.ml.classification.Classifier.getNumClasses(Classifier.scala:115)
	at org.apache.spark.ml.classification.RandomForestClassifier.train(RandomForestClassifier.scala:102)
	at org.apache.spark.ml.classification.RandomForestClassifier.train(RandomForestClassifier.scala:45)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:90)
	at org.apache.spark.ml.Predictor.fit(Predictor.scala:71)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at ja

In [36]:
from pyspark.ml.classification import RandomForestClassifier

In [14]:
from pyspark.ml.classification import LogisticRegression

In [15]:
lr = LogisticRegression()

In [16]:
# Importing the evaluation submodule
import pyspark.ml.evaluation as evals

In [17]:
# Creating a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName = "areaUnderROC")

In [18]:
# Importing the tuning submodule
import pyspark.ml.tuning as tune 

In [19]:
# Create the parameter grid
grid = tune.ParamGridBuilder()

In [22]:
# Add the hyperparameter
grid = grid.addGrid(lr.regParam, np.arange(0, 0.1, 0.01))
grid = grid.addGrid(lr.elasticNetParam, [0,1])

In [23]:
# Build the grid
grid = grid.build()

In [24]:
# Creating the crossvalidator
cv = tune.CrossValidator(estimator = lr,
                        estimatorParamMaps = grid,
                        evaluator = evaluator)

In [44]:
best_lr = lr.fit(train)

IllegalArgumentException: u'Field "label" does not exist.'

In [ ]:
print(best_lr)